# Personal Search 01

Busca de informações em livros usando FAISS

## Import packages

In [1]:
from langchain_community.llms import Ollama
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from operator import itemgetter
from langchain_community.llms import Ollama
from IPython.display import display, Markdown

## Configuration

In [2]:
model = "mistral:7b"
embeddings_model = "nomic-embed-text:v1.5"
path_books = "./vector-store/books"
question = "Como Deus trabalha em nossa vida?"

## Retriever Instance

In [3]:
embeddings = OllamaEmbeddings(model=embeddings_model)

db = FAISS.load_local(path_books, embeddings, allow_dangerous_deserialization=True)

retriever = db.as_retriever()
retriever.search_kwargs["distance_metric"] = "cos"
retriever.search_kwargs["fetch_k"] = 100
retriever.search_kwargs["maximal_marginal_relevance"] = False
retriever.search_kwargs["k"] = 5

## Search

In [4]:
# RAG
template = """
- Retorne apenas citações do que está entre as tags <context>. 
- Retorne até 5 citações.
- Retorne apenas citações relacionada a busca.
- Informe também o nome do livro e autor.
- Não formule resposta própria. 
- Responda no idioma original da pergunta.

<context> 
{context} 
<context>

Busca: {question}

Citações:
"""

prompt = ChatPromptTemplate.from_template(template)

llm = Ollama(model=model, temperature=0)

final_rag_chain = (
    {"context": retriever, 
     "question": itemgetter("question")} 
    | prompt
    | llm
    | StrOutputParser()
)

final_answer = final_rag_chain.invoke({"question":question})

In [5]:
display(Markdown(f"**Pergunta:** {question}\n\n"))
display(Markdown(final_answer))

 "Nadando no amor misericordioso do Cristo redentor, somos livres para rir da tendência humana de assumir ares de superioridade espiritual — até mesmo em nós mesmos. Somos livres para estender aos outros a misericórdia que recebemos." (Evangelho Maltrapilho, p. 73)

"Essa foi nossa postura. Percebemos que as pessoas que nos prejudicaram estavam talvez espiritualmente doentes. Embora não gostássemos dos sintomas e do modo como eles nos afetavam, elas, como nós, estavam também doentes. Pedimos a Deus que nos ajudasse a mostrar a elas a mesma tolerância, compaixão e paciência que mostrariamos com toda disposição a um amigo doente." (Evangelho Maltrapilho, p. 104)

"Quando uma pessoa ofendia, dizíamos a nós mesmos: 'Eis uma pessoa doente. Deus, salva-me de perder a paciência. Seja feita a tua vontade'." (Evangelho Maltrapilho, p. 104)

"Com um Deus cujo nome é Misericórdia, não é de admirar que a gratidão ultrapasse qualquer outra atitude ao longo de nossa jornada espiritual." (Evangelho Maltrapilho, p. 104)

"Não passamos pela vida alheios ao preço pago pela nossa libertação." (Evangelho Maltrapilho, p. 104)